In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from VQLS import getApproximationValue, plotCost, minimization, getMatrixCoeffitients, ansatzTest
from LSSVM import predict, lssvmMatrix, prepareLabels, accuracy

In [2]:
auxiliaryQubit = 0
shots = 10000 # 100000
gamma = 0.01 # regularization parameter
testSize = 0.2

In [3]:
X, y = datasets.make_classification(
    n_samples=50, n_features=2, n_informative=2, n_redundant=0, random_state=45
)
y = np.where(y == 0, -1, 1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=testSize, random_state=42
)
subsetSize = 7
subsetIndices = np.random.choice(np.arange(X_train.shape[0]), subsetSize, replace=False)
xSubset = np.array([X_train[i] for i in subsetIndices])
ySubset = np.array([y_train[i] for i in subsetIndices])

inputMatrix = lssvmMatrix(xSubset, gamma, "linearKernel")
yVector = prepareLabels(ySubset)

print("LS-SVM Matrix:", inputMatrix)

pauliOp = SparsePauliOp.from_operator(inputMatrix)
paulis = pauliOp.paulis
print(paulis)

coefficientSet = getMatrixCoeffitients(pauliOp)
print(coefficientSet)

outF = minimization(
    paulis=paulis,
    coefficientSet=coefficientSet,
    totalNeededQubits=pauliOp.num_qubits + 2,
    bVector=yVector,
    quantumSimulation=True,
)
circ = QuantumCircuit(pauliOp.num_qubits, pauliOp.num_qubits)
o = ansatzTest(circ, outF)

getApproximationValue(inputMatrix, yVector, o)
plotCost()

LS-SVM Matrix: [[ 0.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00
   1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.01875017e+02  5.38214142e-02  4.39809693e-01
   1.73882042e+00  3.90872973e-02  3.43590655e-01  7.11363596e-01]
 [ 1.00000000e+00  5.38214142e-02  1.01715282e+02  1.16699852e+00
  -1.79954494e-01 -2.68690672e+00  1.98189378e+00 -2.63972072e+00]
 [ 1.00000000e+00  4.39809693e-01  1.16699852e+00  1.00880750e+02
   2.53024805e-01 -1.80148896e+00  1.40895479e+00 -1.62501200e+00]
 [ 1.00000000e+00  1.73882042e+00 -1.79954494e-01  2.53024805e-01
   1.01643349e+02  3.96797944e-01  5.41211460e-02  1.01650092e+00]
 [ 1.00000000e+00  3.90872973e-02 -2.68690672e+00 -1.80148896e+00
   3.96797944e-01  1.04217039e+02 -3.08600547e+00  4.18730916e+00]
 [ 1.00000000e+00  3.43590655e-01  1.98189378e+00  1.40895479e+00
   5.41211460e-02 -3.08600547e+00  1.02332217e+02 -2.93072105e+00]
 [ 1.00000000e+00  7.11363596e-01 -2.63972072e+00 -1.6

KeyboardInterrupt: 

In [4]:
print(o)
print(lssvm_matrix.dot(o))

getApproximationValue(lssvm_matrix, yVector, o)
plotCost()

NameError: name 'o' is not defined

In [ ]:
subsetIndicesTest = np.random.choice(
    np.arange(X_test.shape[0]), subsetSize, replace=False
)
xTestSubset = []
yTestSubset = []
for i in subsetIndicesTest:
    xTestSubset.append(X_test[i])
    yTestSubset.append(y_test[i])

xTestSubset = np.array(xTestSubset)
yTestSubset = np.array(yTestSubset)
print(o)
b = o[0]
print(b)
w = []
for i in range(len(o)):
    if i != 0:
        w.append(o[i])
print(w)
predictions = predict(xSubset, xTestSubset, w, b)
print("Predictions:", predictions)
print("Accuracy:", accuracy(yTestSubset, predictions))